## Setup Environment
Python Environment: Use an environment with Python and necessary libraries installed (e.g., numpy, pandas, matplotlib for data manipulation and visualization; TensorFlow or PyTorch for neural network modeling).

In [92]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd



## prepare data

In [97]:
# Assuming 'stock_data' is a DataFrame containing the stock data
file_path = 'TSLA_stock_data_2023.csv'
stock_data = pd.read_csv(file_path)


In [103]:
# Convert date column to datetime if it exists
# if 'Datetime' in stock_data.columns:
#     print("da")
#     stock_data['Date'] = pd.to_datetime(stock_data['Datetime'])
#     stock_data['Year'] = stock_data['Datetime'].dt.year
#     stock_data['Month'] = stock_data['Datetime'].dt.month
#     stock_data['Day'] = stock_data['Datetime'].dt.day
#     # Optionally, drop the original date column if no longer needed
print(stock_data)

# Select only the numeric columns for normalization
numeric_cols = stock_data.select_dtypes(include=['number']).columns
stock_data[numeric_cols] = (stock_data[numeric_cols] - stock_data[numeric_cols].min()) / (stock_data[numeric_cols].max() - stock_data[numeric_cols].min())

stock_data.fillna(method='ffill', inplace=True)  # forward fill to propagate last valid observation forward
print(stock_data[numeric_cols])


                       Datetime      Open      High       Low     Close  \
0     2023-01-03 09:30:00-05:00  0.079293  0.063917  0.043810  0.028010   
1     2023-01-03 10:30:00-05:00  0.041261  0.029894  0.032008  0.014551   
2     2023-01-03 11:30:00-05:00  0.028049  0.010471  0.019441  0.002494   
3     2023-01-03 12:30:00-05:00  0.016139  0.000000  0.014517  0.000000   
4     2023-01-03 13:30:00-05:00  0.013685  0.005051  0.018108  0.008575   
...                         ...       ...       ...       ...       ...   
1737  2023-12-29 11:30:00-05:00  0.743414  0.743068  0.748141  0.742037   
1738  2023-12-29 12:30:00-05:00  0.745643  0.744365  0.748808  0.749065   
1739  2023-12-29 13:30:00-05:00  0.752537  0.749811  0.756861  0.747014   
1740  2023-12-29 14:30:00-05:00  0.750461  0.748151  0.757066  0.752144   
1741  2023-12-29 15:30:00-05:00  0.755510  0.746543  0.750500  0.742192   

        Volume  Dividends  Stock Splits  
0     0.675390        NaN           NaN  
1     0.323906 

In [104]:
# Define the neural network for the agent
def create_model(input_dim):
    print("1")
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        Dense(32, activation='relu'),
        Dense(3, activation='linear')  # Assuming three actions: buy, hold, sell
    ])
    model.compile(optimizer='adam', loss='mse')
    print("1 succ")
    return model


SyntaxError: invalid syntax (568946400.py, line 5)

## Define State and Reward
State Definition: Define the state as a vector of features like the day's opening price, high, low, close, and volume.
Reward Calculation: Calculate rewards based on the change in stock price, as described in the paper.


In [ ]:
# Define the neural network for the agent
def create_model(input_dim):
    print("2")
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        Dense(32, activation='relu'),
        Dense(3, activation='linear')  # Assuming three actions: buy, hold, sell
    ])
    model.compile(optimizer='adam', loss='mse')
    print("2 scs")
    print(model)
    return model


In [ ]:
def choose_action(state, model):
    print("3")
    if isinstance(state, pd.Series):
        state = state.values 

    # Reshape the state to fit the model's input requirements (1, number of features)
    state = state.reshape(1, -1)
    # Use the model to predict the action from the current state
    q_values = model.predict(state)
    print("3 scs")
    return np.argmax(q_values[0])  # Choosing the action with the highest Q-value


In [ ]:
def calculate_reward(current_state, next_state):
    print("4")
    # Assuming 'close_prices' is a list or array of closing prices
    # rewards = np.diff(close_prices) / close_prices[:-1]  # Percentage change between consecutive days
    # return rewards
    current_price = current_state['Close']
    next_price = next_state['Close']
    print("4 scs")
    return (next_price - current_price) / current_price


## Reinforcement Learning Model
Model Initialization: Initialize the parameters for the TD(0) algorithm, including the discount factor (γ) and learning rate (α).
Network Setup: Set up a neural network for function approximation. A simple multi-layer perceptron (MLP) can be used initially.

In [ ]:

def update_model(model, state, action, reward, next_state):
    print("5")
    state = state.values.astype('float32').reshape(1, -1)
    next_state = next_state.values.astype('float32').reshape(1, -1)
    # Perform a TD update on the model
    target = reward + 0.95 * np.amax(model.predict(np.array([next_state]))[0])  # Discount factor gamma = 0.95
    target_vec = model.predict(np.array([state]))[0]
    target_vec[action] = target
    # model.fit(np.array([state]), np.array([target_vec]), epochs=1, verbose=0)
    model.fit(state, target_vec.reshape(-1, 1), epochs=1, verbose=0)
    print("5 scs")


In [ ]:
#Simulate each trading period as an episode. For each episode, reset the environment to an initial state
def run_episode(data, model):
    total_reward = 0
    
    for t in range(len(data) - 1):
        current_state = data[numeric_cols].iloc[t]
        next_state = data[numeric_cols].iloc[t + 1]
        # print(current_state)
        action = choose_action(current_state, model)
        reward = calculate_reward(current_state, next_state)
        
        update_model(model, current_state, action, reward, next_state)
        
        total_reward += reward
    
    return total_reward


## Training the Model
Algorithm: Implement the TD(0) learning algorithm to update the value function based on the state and reward observed from the data.
Iteration: Iterate over episodes (each episode can be a sequence of stock price data), updating the model with each step.

In [ ]:
# Run multiple episodes to train the model effectively.
def train_model(data, model, episodes):
    for e in range(episodes):
        total_reward = run_episode(data, model)
        print(f'Episode {e+1}/{episodes}, Total Reward: {total_reward}')

# Initialize the model
# print(model.input_shape) 
# Initialize the model
num_features = len(stock_data[numeric_cols])  # Update to match the number of input features after preprocessing
model = create_model((None,1742))



2


C:\Users\26210\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Invalid dtype: tuple

In [ ]:

train_model(stock_data, model, 1000)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_12" is incompatible with the layer: expected axis -1 of input shape to have value 1740, but received input with shape (1, 7)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 7), dtype=float32)
  • training=False
  • mask=None

In [ ]:
print(stock_data) 

                       Datetime      Open      High       Low     Close  \
0     2023-01-03 09:30:00-05:00  0.079293  0.063917  0.043810  0.028010   
1     2023-01-03 10:30:00-05:00  0.041261  0.029894  0.032008  0.014551   
2     2023-01-03 11:30:00-05:00  0.028049  0.010471  0.019441  0.002494   
3     2023-01-03 12:30:00-05:00  0.016139  0.000000  0.014517  0.000000   
4     2023-01-03 13:30:00-05:00  0.013685  0.005051  0.018108  0.008575   
...                         ...       ...       ...       ...       ...   
1737  2023-12-29 11:30:00-05:00  0.743414  0.743068  0.748141  0.742037   
1738  2023-12-29 12:30:00-05:00  0.745643  0.744365  0.748808  0.749065   
1739  2023-12-29 13:30:00-05:00  0.752537  0.749811  0.756861  0.747014   
1740  2023-12-29 14:30:00-05:00  0.750461  0.748151  0.757066  0.752144   
1741  2023-12-29 15:30:00-05:00  0.755510  0.746543  0.750500  0.742192   

        Volume  Dividends  Stock Splits  
0     0.675390        NaN           NaN  
1     0.323906 

## Evaluation
Testing: After training, test the model on unseen data to assess its predictive accuracy.
Performance Metrics: Use metrics like RMSE or predictive accuracy grades as used in the paper to evaluate performance.

## Monitoring and Adjustment
Continuous Monitoring: Set up scripts to monitor the model’s performance over time.
Adjustment: Tune parameters and refine the model as needed based on performance metrics.